In [ ]:
import sys
print(sys.version)

In [ ]:
#install packages
!pip install spacy
!pip install xgboost
!pip install en_core_web_md-3.7.1-py3-none-any.whl
!pip install imblearn
!pip3 install imblearn

In [ ]:
#perform imports and load dataset
import numpy as np
import pandas as pd
import scipy
import spacy
import re # For regular expressions|||
import string # For handling string
import nltk
import pickle
import sklearn
import vaderSentiment
import matplotlib.pyplot as plt
from nltk.util import ngrams
#import seaborn as sns
from nltk.corpus import stopwords
from collections import  Counter
#import gensim
#nltk.download('punkt')
#nltk.download('wordnet')
from matplotlib import pyplot
from numpy import array
#from textblob import TextBlob
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from scipy.sparse import csr_matrix
#from imblearn import under_sampling, over_sampling
#from imblearn.datasets import make_imbalance
#from imblearn.over_sampling import SMOTE
#libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
#full labeled data set 
data = pd.read_csv("LabData.csv")

In [ ]:
#data preprocessing:
df1 = data
#CLEANING DATA
#drop duplicates
df1 = df1.drop_duplicates(keep='first')
#removing numbers
df1['text'] = df1['text'].astype('string')
df1.dropna(subset=['text'], inplace=True)
#lowercase
df1['text']=df1['text'].apply(lambda x: x.lower())
#removing mentions
df1['text']=df1['text'].apply(lambda x: re.sub("@[A-Za-z0-9_]+",'', x))
#removing numbers and words containing numbers
df1['text']=df1['text'].apply(lambda x: re.sub('\w*\d\w*','', x))
#removing hashtags?
#df1['text']=df1['text'].apply(lambda x: re.sub("#[A-Za-z0-9_]+",'', x))
#df1['text']=df1['text'].apply(lambda x: re.split('#|_', x))
df1['text']=df1['text'].apply(lambda x: " ".join(word.strip() for word in re.split('#|_', x)))
#removing links
df1['text']=df1['text'].apply(lambda x: re.sub(r"http\S+",'', x))
df1['text']=df1['text'].apply(lambda x: re.sub(rwww.\S+,'', x))
#removing punctuation
df1['text']=df1['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
#removing empty spaces and special characters
df1['text']=df1['text'].apply(lambda x: re.sub('[^A-Za-z0-9]+',' ',x))
#stopwords
nlp = spacy.load('en_core_web_md')
# Print the set of spaCy's default stop words (remember that sets are unordered):
print(nlp.Defaults.stop_words)
#add twitter as a stop word and rt
# Add the word to the set of stop words. Use LOWERCASE!
nlp.Defaults.stop_words.add('twitter')
nlp.Defaults.stop_words.add('rt')
nlp.Defaults.stop_words.add('s')
nlp.Defaults.stop_words.add('tweet')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('follower')
nlp.Defaults.stop_words.add('null')
nlp.Defaults.stop_words.add('http')
nlp.Defaults.stop_words.add('url')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('trending')
# Set the stop_word tag on the lexeme
nlp.vocab['twitter'].is_stop = True
nlp.vocab['rt'].is_stop = True
nlp.vocab['s'].is_stop = True
nlp.vocab['tweet'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['follower'].is_stop = True
nlp.vocab['null'].is_stop = True
nlp.vocab['http'].is_stop = True
nlp.vocab['url'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['trending'].is_stop = True
stop = nlp.Defaults.stop_words
#lemmatisation
# Loading model
#nlp = spacy.load('en_core_web_sm')

In [ ]:
# Lemmatization with stopwords removal
#df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False)&((token.pos_=="ADJ")|(token.pos_=="NOUN")|(token.pos_=="VERB")))]))
#df1.head()

In [ ]:
df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False))]))

In [ ]:
#getting rid of empty text rows
df1 = df1.dropna(subset=['text'])

In [ ]:
data = df1

In [ ]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
# Fit and transform the training data
X_tfidf = vectorizer.fit_transform(data['text'])
X_train_tfidf = X_tfidf.toarray()
#checking type
#test.dtypes()
DF = pd.DataFrame(data=X_train_tfidf, columns=vectorizer.get_feature_names_out())
DF.head
#X_tfidf = X_tfidf.fillna(0)

In [ ]:
# Merge the two DataFrames
DF['text'] = data['text']
DF['label'] = data['label']

In [ ]:
del data

In [ ]:
reducedDF = DF.sample(frac=0.5, random_state=1)

In [ ]:
del DF

In [ ]:
#create train set
#divide into training, validation and test data (DF)
from sklearn.model_selection import train_test_split
ind = reducedDF.index
#sample the indices and then drop from current data set
# Assuming you have features X and labels y
df_train, df_test= train_test_split(ind, test_size=0.5, random_state=42)

In [ ]:
#now create the data sets
TrainSet = reducedDF.drop(df_test)

In [ ]:
del reducedDF

#### Clean the response variable

TrainSet.head

In [ ]:
TrainSet['label']

In [ ]:
mapping = {'-2': -1, '-1': -1, '0': 0, '1': 1, '9': 0, 'FALSE': 0, 'TRUE': 0, 'lab': 0}
TrainSet['label'] = TrainSet['label'].map(mapping)
#TrainSet.fillna(0)

In [ ]:
#training set
#TrainSet = ResponseClean(TrainSet)

In [ ]:
TrainSet['label'].unique

In [ ]:
del TrainSet['text']

In [ ]:
TrainSet.shape

In [ ]:
#split into train and test set
#training set
y_train = TrainSet["label"]

In [ ]:
y_train = y_train.fillna(0)

In [ ]:
y_train.unique()

In [ ]:
del TrainSet['label']

#### Fit the model

In [ ]:
# libraries
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from sklearn.metrics import confusion_matrix
 
# Random forest classifier
# Feature selection
chi2_features = SelectKBest(score_func=chi2, k=1000)
ChiX_selected = chi2_features.fit_transform(TrainSet, y_train)
support = chi2_features.get_support()
 
# Get the selected feature names using the support indices
selected_feature_names = TrainSet.columns[support]
X_selected = TrainSet[selected_feature_names]
 
# Random Oversampling
ros = RandomOverSampler(random_state=0)
X_oversampled1, y_oversampled1 = ros.fit_resample(X_selected, y_train)
 
# Fit the Random Forest model to the training data
RFC1 = RandomForestClassifier(n_estimators=50, max_depth=10)
RFC1.fit(X_oversampled1, y_oversampled1)

#### Need to get the output per data set

In [ ]:
OGdata = pd.read_csv("ConsFinal.csv")
#due to size had to split it up

In [ ]:
OGdata.shape

In [ ]:
#divide the data
del data1
data1 = OGdata[800000:]

#### Data (1,2,3,4,5)
 

In [ ]:
#data preprocessing:
df1 = data1
#CLEANING DATA
#drop duplicates
df1 = df1.drop_duplicates(keep='first')
#removing numbers
df1['text'] = df1['text'].astype('string')
df1.dropna(subset=['text'], inplace=True)
#lowercase
df1['text']=df1['text'].apply(lambda x: x.lower())
#removing mentions
df1['text']=df1['text'].apply(lambda x: re.sub("@[A-Za-z0-9_]+",'', x))
#removing numbers and words containing numbers
df1['text']=df1['text'].apply(lambda x: re.sub('\w*\d\w*','', x))
#removing hashtags?
#df1['text']=df1['text'].apply(lambda x: re.sub("#[A-Za-z0-9_]+",'', x))
#df1['text']=df1['text'].apply(lambda x: re.split('#|_', x))
df1['text']=df1['text'].apply(lambda x: " ".join(word.strip() for word in re.split('#|_', x)))
#removing links
df1['text']=df1['text'].apply(lambda x: re.sub(r"http\S+",'', x))
df1['text']=df1['text'].apply(lambda x: re.sub(rwww.\S+,'', x))
#removing punctuation
df1['text']=df1['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
#removing empty spaces and special characters
df1['text']=df1['text'].apply(lambda x: re.sub('[^A-Za-z0-9]+',' ',x))
#stopwords
nlp = spacy.load('en_core_web_md')
# Print the set of spaCy's default stop words (remember that sets are unordered):
print(nlp.Defaults.stop_words)
#add twitter as a stop word and rt
# Add the word to the set of stop words. Use LOWERCASE!
nlp.Defaults.stop_words.add('twitter')
nlp.Defaults.stop_words.add('rt')
nlp.Defaults.stop_words.add('s')
nlp.Defaults.stop_words.add('tweet')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('follower')
nlp.Defaults.stop_words.add('null')
nlp.Defaults.stop_words.add('http')
nlp.Defaults.stop_words.add('url')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('trending')
# Set the stop_word tag on the lexeme
nlp.vocab['twitter'].is_stop = True
nlp.vocab['rt'].is_stop = True
nlp.vocab['s'].is_stop = True
nlp.vocab['tweet'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['follower'].is_stop = True
nlp.vocab['null'].is_stop = True
nlp.vocab['http'].is_stop = True
nlp.vocab['url'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['trending'].is_stop = True
stop = nlp.Defaults.stop_words
#lemmatisation
# Loading model
#nlp = spacy.load('en_core_web_sm')

In [ ]:
# Lemmatization with stopwords removal
#df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False)&((token.pos_=="ADJ")|(token.pos_=="NOUN")|(token.pos_=="VERB")))]))
#df1.head()
df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False))]))

In [ ]:
#getting rid of empty text rows
df1 = df1.dropna(subset=['text'])

In [ ]:
data = df1

In [ ]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
# Fit and transform the training data
X_tfidf = vectorizer.fit_transform(data['text'])
X_train_tfidf = X_tfidf.toarray()
#checking type
#test.dtypes()
DF = pd.DataFrame(data=X_train_tfidf, columns=vectorizer.get_feature_names_out())
DF.head
#X_tfidf = X_tfidf.fillna(0)

In [ ]:
# equate the column variables
test = pd.DataFrame(DF)
test_cols = test.columns
common_cols = selected_feature_names.intersection(test_cols)
testnew = test[common_cols]
testnew = testnew.reindex(columns=selected_feature_names, fill_value=0)
del y_pred1
# Predict the labels of the testing data
y_pred1 = RFC1.predict(testnew)

In [ ]:
len(y_pred1) #sense check
 

In [ ]:
data['y_values'] = y_pred1

In [ ]:
RF = data
RF.to_csv("RF9.csv")
del DF
del X_tfidf